# AST325/326

### In this notebook we will go over peak and centroid finding in Python!

In [ ]:
#import everything
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import numpy as np
import scipy as sp

Let's load a dataset of a xenon spectrum. (NIST Atomic Spectra Database, https://www.atomtrace.com/elements-database/element/54)

In [ ]:
wavelength, intensity = np.loadtxt('54_NIST.asc', unpack = True)

In [ ]:
plt.rcParams["figure.figsize"] = (7,4)
plt.rcParams['font.size'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Let's take a look at the data.

In [ ]:
plt.figure()

plt.plot(wavelength, intensity, color = 'purple')

plt.xlabel('Wavelength [nm]')
plt.ylabel('Intensity [a.u.]')

plt.grid(ls = '-.')
plt.show()

### Now let's use scipy.find_peaks to find the peak locations. We have multiple parameters we can use to specify how many/what types of peaks we want. Play around with different settings to compare them!

* **Height:** Required height of peaks. Either a number, None, an array matching x or a 2-element sequence of the former. The first element is always interpreted as the minimal and the second, if supplied, as the maximal required height.

* **Width:** Required width of peaks in samples. Either a number, None, an array matching x or a 2-element sequence of the former. The first element is always interpreted as the minimal and the second, if supplied, as the maximal required width.

* **Distance:** Required minimal horizontal distance (>= 1) in samples between neighbouring peaks. Smaller peaks are removed first until the condition is fulfilled for all remaining peaks.

* **Threshold:** Required threshold of peaks, the vertical distance to its neighboring samples. Either a number, None, an array matching x or a 2-element sequence of the former. The first element is always interpreted as the minimal and the second, if supplied, as the maximal required threshold.

* **Prominence:** Required prominence of peaks. Either a number, None, an array matching x or a 2-element sequence of the former. The first element is always interpreted as the minimal and the second, if supplied, as the maximal required prominence.



In [ ]:
peaks = sp.signal.find_peaks(intensity, prominence = 7e-3)[0]

num_peaks = len(peaks)

In [ ]:
plt.figure()

plt.plot(wavelength, intensity, color = 'purple', label = 'Data')
plt.plot(wavelength[peaks], intensity[peaks], color = 'coral', ls = '',
         marker = 'x',  ms = 5, label = 'Peaks')

plt.xlabel('Wavelength [nm]')
plt.ylabel('Intensity [a.u.]')

plt.legend(loc = 'best', fontsize = 10)
plt.grid(ls = '-.')
plt.show()

#### We can now use these peaks to find the centroids which can be thought of as the "center of mass" or weighted mean of the peaks. The peaks may not be the actual peak value as we are limited by our measurments which is why we we can't just use the peak value!

Recall,

$x_{\text{cent}} = \frac{∑x_iI_i}{∑I_i}$,

where $x_{\text{cent}}$ is the centroid, $x_i$ are the x-values, and $I_i$ are the intensity values. Also note that this assumes that the background is distributed around 0!

In [ ]:
def centroid(x, I):
  '''
  Returns the centroid value for a given set of x- and intensity values.

  Input: x: x values, I: intensity values
  Output: centroid value
  '''

  return np.sum(x*I)/np.sum(I)

Let's look at what value we should use for our centroid ranges around the peaks.

In [ ]:
plt.figure()

range_val = 50

colours = cm.plasma(np.linspace(0, 1, num_peaks))

plt.plot(wavelength, intensity, color = 'purple', label = 'Data')
plt.plot(wavelength[peaks], intensity[peaks], color = 'coral', ls = '',
         marker = 'x',  ms = 5, label = 'Peaks')

[plt.axvline(wavelength[peaks[i]-range_val], color = colours[i], alpha = 0.5)
for i in range(len(peaks))]

[plt.axvline(wavelength[peaks[i]+range_val], color = colours[i], alpha = 0.5)
for i in range(len(peaks))]

plt.xlabel('Wavelength [nm]')
plt.ylabel('Intensity [a.u.]')

plt.legend(loc = 'best', fontsize = 10)
plt.grid(ls = '-.')
plt.xlim((400, 800))
plt.show()

Let's zoom in to confirm these ranges.

In [ ]:
plt.figure()

colours_t = cm.plasma(np.linspace(0, 1, 10))

plt.plot(wavelength, intensity, color = 'purple', label = 'Data')
plt.plot(wavelength[peaks], intensity[peaks], color = 'coral', ls = '',
         marker = 'x',  ms = 5, label = 'Peaks')

[plt.axvline(wavelength[peaks[i]-range_val], color = colours_t[i], alpha = 0.5)
for i in range(len(peaks[:10]))]

[plt.axvline(wavelength[peaks[i]+range_val], color = colours_t[i], alpha = 0.5)
for i in range(len(peaks[:10]))]

plt.xlabel('Wavelength [nm]')
plt.ylabel('Intensity [a.u.]')

plt.legend(loc = 'best', fontsize = 10)
plt.grid(ls = '-.')
plt.xlim((425, 500))
plt.show()

In [ ]:
centroids = [centroid(wavelength[peaks[i]-range_val:peaks[i]+range_val], intensity[peaks[i]-range_val:peaks[i]+range_val]) for i in range(len(peaks))]

Let's plot the centroids.

In [ ]:
fig = plt.figure()

gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1], hspace = 0.0)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharex = ax0)

ax0.plot(wavelength, intensity, color = 'purple', label = 'Data')
ax0.plot(wavelength[peaks], intensity[peaks], color = 'coral', ls = '', marker = 'x',
         ms = 5, label = 'Peaks')

[ax0.axvline(c, color = 'teal', alpha = 0.5, label = 'Centroids') if c == centroids[0]
 else ax0.axvline(c, color = 'teal', alpha = 0.5) for c in centroids]

ax1.plot(wavelength[peaks], wavelength[peaks] - centroids, ls = '', marker = '.', color = 'coral')

ax1.set_xlabel('Wavelength [nm]')
ax0.set_ylabel('Intensity [a.u.]')
ax1.set_ylabel('Residuals')

plt.setp(ax0.get_xticklabels(), visible = False)

ax0.legend(loc = 'best', fontsize = 10)
ax0.grid(ls = '-.')
ax1.grid(ls = '-.')
plt.xlim((350, 850))
plt.show()

Let's zoom in to check!

In [ ]:
fig = plt.figure()

gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1], hspace = 0.0)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharex = ax0)

ax0.plot(wavelength, intensity, color = 'purple', label = 'Data')
ax0.plot(wavelength[peaks], intensity[peaks], color = 'coral', ls = '', marker = 'x',
         ms = 5, label = 'Peaks')

[ax0.axvline(c, color = 'teal', alpha = 0.5, label = 'Centroids') if c == centroids[0]
 else ax0.axvline(c, color = 'teal', alpha = 0.5) for c in centroids]

ax1.plot(wavelength[peaks], wavelength[peaks] - centroids, ls = '', marker = '.', color = 'coral')

ax1.set_xlabel('Wavelength [nm]')
ax0.set_ylabel('Intensity [a.u.]')
ax1.set_ylabel('Residuals')

plt.setp(ax0.get_xticklabels(), visible = False)

ax0.legend(loc = 'best', fontsize = 10)
ax0.grid(ls = '-.')
ax1.grid(ls = '-.')
plt.xlim(425, 500)
plt.show()